In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My\ Drive/

/content/drive/My Drive


In [0]:
%%bash
#git clone https://github.com/cdqa-suite/cdQA.git 

In [3]:
%cd cdQA 

/content/drive/My Drive/cdQA


In [0]:
!ls

In [4]:
%%bash
pip install -r 'requirements.txt'
pip install .
pip install 'pytorch_pretrained_bert' #for gpu version maybe

  Created wheel for tika: filename=tika-1.19-cp36-none-any.whl size=29224 sha256=301d75c33cccca76d26597373e01f917452785584651b84c4726e5aee5e41d96
  Stored in directory: /root/.cache/pip/wheels/b4/db/8a/3a3f0c0725448eaa92703e3dda71e29dc13a119ff6c1036848
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=468ae96428ff734def7a2b2114955dcb3c49e21e91e9dd713f972a36dbfb965d
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=4b6c9a9ad032daedf6ca8e6effbe5a4fe92e287b7ea35f63fffbbb86cd5a7d60
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built tika wget sacremoses
  Found existing installation: joblib 0.14.0
    Uninstalling joblib-0.14.0:
      Successfully uninstalled joblib-0.14.0
  Found existing installation: pandas 0.25.3
    Uninstalling pand

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('punkt')

In [4]:
import pandas as pd
from ast import literal_eval
from cdqa.utils.filters import filter_paragraphs
from cdqa.utils.download import download_model #, download_bnpp_data #not now
from cdqa.pipeline.cdqa_sklearn import QAPipeline
dir='/content/drive/My Drive/cdQA/data'
download_model(model='bert-squad_1.1', dir=dir) 
#download_bnpp_data(dir=dir) #not now

/usr/local/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [33]:
from nltk import tokenize

def load_from_csv(file):
    df = pd.read_csv(file)
    # column name has to be in lowercase
    df = df.rename(str.lower, axis='columns')
    df['paragraphs'] = df['paragraphs'].apply(lambda x: x.replace("'s", " " "s").replace("\n"," "))
    # split text by sentences
    df['paragraphs'] = df['paragraphs'].apply(lambda x: tokenize.sent_tokenize(x))
    return df

In [34]:
df = load_from_csv('./data/test.csv')
df.head()

,title,paragraphs
0,Chisholm versus Lindsay,"[On April 22, 2005 just before noon, the Plain..."


In [36]:
cdqa_pipeline = QAPipeline(reader='bert_qa_vCPU-sklearn.joblib')
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(bert_model='bert-base-uncased', do_lower_case=True, fp16=False,
    gradient_accumulation_steps=1, learning_rate=3e-05, local_rank=-1,
    loss_scale=0, max_answer_length=30, n_best_size=20, no_cuda=False,
    null_score_diff_threshold=0.0, num_train_epochs=2, output_dir=None,
    predict_batch_size=8, seed=42, server_ip='', server_port='',
    train_batch_size=8, verbose_logging=False,
    version_2_with_negative=False, warmup_proportion=0.1),
      retrieve_by_doc=False,
      retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True, max_df=0.85,
       min_df=2, ngram_range=(1, 2), preprocessor=None,
       stop_words='english', token_pattern='(?u)\\b\\w\\w+\\b',
       tokenizer=None, top_n=20, verbose=False, vocabulary=None))

In [29]:
import os
from nltk import tokenize

def clean_text(text):
    return text.replace("'s", " " "s").replace("\n"," ")


def process_txt_data(path):
    data = {'title': [], 
            'paragraphs': []}
    for filename in os.listdir(path):
        if filename.endswith(".txt"): 
            with open(path+filename, 'r') as f:
                data['title'].append(filename)
                data['paragraphs'].append(clean_text(f.read()))
    df = pd.DataFrame(data=data)
    df['paragraphs'] = df['paragraphs'].apply(lambda x: tokenize.sent_tokenize(x))
    return df

In [0]:
'''
# load pretrained BERT model
cdqa_pipeline = QAPipeline(reader='bert_qa_vCPU-sklearn.joblib')

# prepare corpus
# enter directory path (e.g. "data/")
df = process_txt_data(path="./data/test.csv")

# fit corpus to model
cdqa_pipeline.fit_retriever(df=df)
'''

'\n# load pretrained BERT model\ncdqa_pipeline = QAPipeline(reader=\'bert_qa_vCPU-sklearn.joblib\')\n\n# prepare corpus\n# enter directory path (e.g. "data/")\ndf = process_txt_data(path="./data/bnpp_newsroom-v1.1.csv")\n\n# fit corpus to model\ncdqa_pipeline.fit_retriever(df=df)\n'

In [0]:
#! usr/bin/env python
cdqa_pipeline.predict(query='Since when does the Excellence Program of BNP Paribas exist?')

('January 2016',
 'BNP Paribas’ commitment to universities and schools',
 'Since January 2016, BNP Paribas has offered an Excellence Program targeting new Master’s level graduates (BAC+5) who show high potential. The aid program lasts 18 months and comprises three assignments of six months each. It serves as a strong career accelerator that enables participants to access high-level management positions at a faster rate. The program allows participants to discover the BNP Paribas Group and its various entities in France and abroad, build an internal and external network by working on different assignments and receive personalized assistance from a mentor and coaching firm at every step along the way.',
 16.854050833952467)